# LDA

Instalasi beberapa library yang diperlukan, untuk NLTK dan GENSIM sudah include di Colab

In [ ]:
pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
pip install pyldavis==3.2.1

     |████████████████████████████████| 1.7 MB 25.5 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136187 sha256=6526fde65ff1dea1da973659d5d80d9444734076739be44b7e0dfb80317820fc
  Stored in directory: /root/.cache/pip/wheels/c6/ee/a6/7c17a63623f940dff0b9cbd7e48a27543f088fa55a7d2b62d0
Successfully built pyldavis


In [ ]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 24.1 MB/s 


In [ ]:
pip install swifter

     |████████████████████████████████| 647 kB 27.1 MB/s 
     |████████████████████████████████| 280 kB 64.8 MB/s 
     |████████████████████████████████| 136 kB 58.4 MB/s 
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=4aa46e84433d60bf774516503588a877a19ba40d0132c984f314dfbb08c7ad46
  Stored in directory: /root/.cache/pip/wheels/e8/1a/1b/46aa7b0fac16d3ce1bccaac14bfe4fad972642217ae78d8160
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# Preparing Data

In [ ]:
import pandas as pd 
import numpy as np
import nltk

In [ ]:
dataSB = pd.read_csv('/cleaning_data_final.csv')

dataSB.head()

,tweet
0,ape ye kata linjur tinggal komat kamit lolos o...
1,ptm alesannya dapet uang jajan gampang msi nge...
2,this lumayan rajin ngumpul tugas ofc caper gur...
3,hikss samaa puas waktu jawabin tremor jdi gagu...
4,ngapalin nyatet kadang kadang


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataSB['tweet'] = dataSB['tweet'].str.lower()

print('Case Folding Result : \n')
print(dataSB['tweet'].head(5))

Case Folding Result : 

0    ape ye kata linjur tinggal komat kamit lolos o...
1    ptm alesannya dapet uang jajan gampang msi nge...
2    this lumayan rajin ngumpul tugas ofc caper gur...
3    hikss samaa puas waktu jawabin tremor jdi gagu...
4                        ngapalin nyatet kadang kadang
Name: tweet, dtype: object


# Tokenizing

Menghapus karakter-karakter yang tidak penting dari text

In [ ]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [ ]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
dataSB['tweet'] = dataSB['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataSB['tweet'] = dataSB['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataSB['tweet'] = dataSB['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['tweet'] = dataSB['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataSB['tweet'] = dataSB['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataSB['tweet'] = dataSB['tweet'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataSB['tweet_tokens'] = dataSB['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(dataSB['tweet_tokens'].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Tokenizing Result : 

0    [ape, ye, kata, linjur, tinggal, komat, kamit,...
1    [ptm, alesannya, dapet, uang, jajan, gampang, ...
2    [this, lumayan, rajin, ngumpul, tugas, ofc, ca...
3    [hikss, samaa, puas, waktu, jawabin, tremor, j...
4                   [ngapalin, nyatet, kadang, kadang]
Name: tweet_tokens, dtype: object


In [ ]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

dataSB['tweet_tokens_fdist'] = dataSB['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(dataSB['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(ape, 1), (ye, 1), (kata, 1), (linjur, 1), (t...
1    [(guru, 2), (nilai, 2), (ptm, 1), (alesannya, ...
2    [(this, 1), (lumayan, 1), (rajin, 1), (ngumpul...
3    [(hikss, 1), (samaa, 1), (puas, 1), (waktu, 1)...
4            [(kadang, 2), (ngapalin, 1), (nyatet, 1)]
Name: tweet_tokens_fdist, dtype: object


# Stopwords
Membersihkan teks dari kata-kata yang termasuk ke dalam stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'bisnis', 'pandemi', 'indonesia'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['tweet_tokens_WSW'] = dataSB['tweet_tokens'].apply(stopwords_removal) 

print(dataSB['tweet_tokens_WSW'].head())

0    [ape, ye, linjur, tinggal, komat, kamit, lolos...
1    [ptm, alesannya, dapet, uang, jajan, gampang, ...
2    [this, lumayan, rajin, ngumpul, tugas, ofc, ca...
3    [hikss, samaa, puas, jawabin, tremor, jdi, gag...
4                   [ngapalin, nyatet, kadang, kadang]
Name: tweet_tokens_WSW, dtype: object


In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in dataSB['tweet_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))


6673


In [ ]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
print(term,":" ,term_dict[term])

sedep : sedep


In [ ]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

dataSB['tweet_tokens_stemmed'] = dataSB['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)

print(dataSB['tweet_tokens_stemmed'])

Pandas Apply:   0%|          | 0/3578 [00:00<?, ?it/s]

0       [ape, ye, linjur, tinggal, komat, kamit, lolos...
1       [ptm, alesannya, dapet, uang, jajan, gampang, ...
2       [this, lumayan, rajin, ngumpul, tugas, ofc, ca...
3       [hikss, samaa, puas, jawabin, tremor, jdi, gag...
4                      [ngapalin, nyatet, kadang, kadang]
                              ...                        
3573                           [besok, kimia, apaaannnnn]
3574    [desain, grafis, kemarin, niat, bentrok, buang...
3575                         [ngerasa, salah, hr, latsol]
3576    [join, panitia, volunteer, beban, berat, maksi...
3577                        [tani, anak, anak, sd, sedep]
Name: tweet_tokens_stemmed, Length: 3578, dtype: object


In [ ]:
#stopwords #2

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["ada", "tan", "ton", "pt", "komentar", "juta", "unit", "menang", "artikel", 
                       "smartphone", "tagar", "sedia", "kaskus", "seksi"])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['tweet_tokens_stemmed2'] = dataSB['tweet_tokens_stemmed'].apply(stopwords_removal) 

print(dataSB['tweet_tokens_stemmed2'].head())

0    [ape, ye, linjur, tinggal, komat, kamit, lolos...
1    [ptm, alesannya, dapet, uang, jajan, gampang, ...
2    [this, lumayan, rajin, ngumpul, tugas, ofc, ca...
3    [hikss, samaa, puas, jawabin, tremor, jdi, gag...
4                   [ngapalin, nyatet, kadang, kadang]
Name: tweet_tokens_stemmed2, dtype: object


In [ ]:
doc_clean = dataSB['tweet_tokens_stemmed2']
doc_clean

0       [ape, ye, linjur, tinggal, komat, kamit, lolos...
1       [ptm, alesannya, dapet, uang, jajan, gampang, ...
2       [this, lumayan, rajin, ngumpul, tugas, ofc, ca...
3       [hikss, samaa, puas, jawabin, tremor, jdi, gag...
4                      [ngapalin, nyatet, kadang, kadang]
                              ...                        
3573                           [besok, kimia, apaaannnnn]
3574    [desain, grafis, kemarin, niat, bentrok, buang...
3575                         [ngerasa, salah, hr, latsol]
3576    [join, panitia, volunteer, beban, berat, maksi...
3577                        [tani, anak, anak, sd, sedep]
Name: tweet_tokens_stemmed2, Length: 3578, dtype: object

# LDA model using gensim
Proses topic modeling dengan LDA gensim

In [ ]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<6665 unique tokens: ['ape', 'bestie', 'gatau', 'gila', 'kamit']...>


In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

In [ ]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.016*"latsol" + 0.010*"baca" + 0.010*"tugas" + 0.007*"minggu" + 0.006*"besok" + 0.006*"to" + 0.006*"well" + 0.006*"guru" + 0.006*"rest" + 0.005*"fokus"'),
 (1,
  '0.010*"tugas" + 0.009*"jam" + 0.007*"besok" + 0.007*"suka" + 0.007*"susah" + 0.006*"kasih" + 0.006*"ajar" + 0.006*"kuliah" + 0.006*"buku" + 0.006*"ngerti"'),
 (2,
  '0.011*"latsol" + 0.011*"soshum" + 0.011*"anak" + 0.009*"linjur" + 0.009*"ajar" + 0.007*"minggu" + 0.007*"materi" + 0.006*"ipa" + 0.006*"sma" + 0.006*"jurus"')]

In [ ]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count']) 
print(df_imp_wcount)

      word  topic_id  importance  word_count
0   latsol         0    0.016093         307
1     baca         0    0.009993         167
2    tugas         0    0.009848         238
3   minggu         0    0.006975         172
4    besok         0    0.006268         171
5       to         0    0.005883         143
6     well         0    0.005855          61
7     guru         0    0.005546         112
8     rest         0    0.005511          58
9    fokus         0    0.004848          85
10   tugas         1    0.010268         238
11     jam         1    0.009442         151
12   besok         1    0.006865         171
13    suka         1    0.006773         111
14   susah         1    0.006589         145
15   kasih         1    0.006370          79
16    ajar         1    0.005771         236
17  kuliah         1    0.005635         133
18    buku         1    0.005618         141
19  ngerti         1    0.005615          71
20  latsol         2    0.010950         307
21  soshum

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             0.0              0.5309   
1            1             1.0              0.9663   
2            2             1.0              0.9441   
3            3             2.0              0.8533   
4            4             1.0              0.5209   
5            5             0.0              0.8281   
6            6             1.0              0.9446   
7            7             2.0              0.6748   
8            8             1.0              0.7109   
9            9             1.0              0.9519   

                                            Keywords  \
0  latsol, baca, tugas, minggu, besok, to, well, ...   
1  tugas, jam, besok, suka, susah, kasih, ajar, k...   
2  tugas, jam, besok, suka, susah, kasih, ajar, k...   
3  latsol, soshum, anak, linjur, ajar, minggu, ma...   
4  tugas, jam, besok, suka, susah, kasih, ajar, k...   
5  latsol, baca, tugas, minggu, besok, to, well, ...   
6  tugas, jam

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

In [ ]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [ ]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [ ]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.108202  0.099075       1        1  35.953604
1      0.035160 -0.139611       2        1  34.975000
0     -0.143361  0.040536       3        1  29.071396, topic_info=        Term        Freq       Total Category  logprob  loglift
130   latsol  253.000000  253.000000  Default  30.0000  30.0000
1287  soshum  114.000000  114.000000  Default  29.0000  29.0000
7     linjur   97.000000   97.000000  Default  28.0000  28.0000
532     anak  129.000000  129.000000  Default  27.0000  27.0000
12     tugas  193.000000  193.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
771     buku   23.745741  117.261164   Topic3  -5.8837  -0.3616
458       us   21.016168   60.735612   Topic3  -6.0058   0.1742
148     coba   20.962326   68.445729   Topic3  -6.0084   0.0521
1037  review   20.459327   44.529256   Topic3  -6.0326   0.4577
198   kuliah   20.766080  110.304412   Topic3  -6.0178  -0.4345

[201 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
443       1  0.490278   abis
443       2  0.270943   abis
443       3  0.245139   abis
1436      3  0.923560     ah
157       1  0.488085   ajar
...     ...       ...    ...
458       1  0.576268     us
458       2  0.082324     us
458       3  0.345761     us
303       3  0.989998   well
1143      1  0.980654  worth

[259 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [ ]:
start=1
limit=21
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step)
#show graphs
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: ignored